In [1]:
import datapackage
import pandas as pd

In [2]:
data = pd.read_csv('https://pkgstore.datahub.io/sports-data/atp-world-tour-tennis-data/rankings_1973-2017/archive/03b54038cbe7a33b43af946f34561df2/rankings_1973-2017.csv',
                  low_memory=False)

In [3]:
data.head()

,week_title,week_year,week_month,week_day,rank_text,rank_number,move_positions,move_direction,player_age,ranking_points,tourneys_played,player_url,player_slug,player_id
0,2017.11.20,2017,11,20,1,1,NaN,NaN,31.0,10645,18,/en/players/rafael-nadal/n409/overview,rafael-nadal,n409
1,2017.11.20,2017,11,20,2,2,NaN,NaN,36.0,9605,17,/en/players/roger-federer/f324/overview,roger-federer,f324
2,2017.11.20,2017,11,20,3,3,3.0,up,26.0,5150,23,/en/players/grigor-dimitrov/d875/overview,grigor-dimitrov,d875
3,2017.11.20,2017,11,20,4,4,1.0,down,20.0,4610,25,/en/players/alexander-zverev/z355/overview,alexander-zverev,z355
4,2017.11.20,2017,11,20,5,5,1.0,down,24.0,4015,27,/en/players/dominic-thiem/tb69/overview,dominic-thiem,tb69


In [4]:
df = data[['week_year','player_age','ranking_points','tourneys_played','player_id','player_slug','rank_number']]

In [5]:
df = df[df['rank_number']>50]
import numpy as np
df = df[np.where([df.week_year != 2017] and [df.rank_number > 100],True,False).transpose()]

In [6]:
df

,week_year,player_age,ranking_points,tourneys_played,player_id,player_slug,rank_number
100,2017,33.0,571,31,sc73,rogerio-dutra%20silva,101
101,2017,30.0,570,25,k834,blaz-kavcic,102
102,2017,32.0,564,21,b837,marcos-baghdatis,103
103,2017,35.0,561,22,m873,nicolas-mahut,104
104,2017,20.0,558,25,fb98,taylor-fritz,105
...,...,...,...,...,...,...,...
2694534,1973,26.0,0,0,h064,humphrey-hose,181
2694535,1973,23.0,0,0,r096,franklin-robbins,182
2694536,1973,25.0,0,0,l083,david-lloyd,183
2694537,1973,29.0,0,0,s146,nicola%20-spear,184


In [7]:
df = df[['player_id','player_age','ranking_points','tourneys_played']]
df = df[df['tourneys_played']!=0]
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [8]:
df

,player_id,player_age,ranking_points,tourneys_played
100,sc73,33.0,571,31
101,k834,30.0,570,25
102,b837,32.0,564,21
103,m873,35.0,561,22
104,fb98,20.0,558,25
...,...,...,...,...
1959953,t255,18.0,0,1
1959957,t139,32.0,0,1
1959968,w246,20.0,0,1
1959970,w255,19.0,0,1


In [9]:
X = df.drop('ranking_points',axis=1)
y = df['ranking_points']

In [10]:
X

,player_id,player_age,tourneys_played
100,sc73,33.0,31
101,k834,30.0,25
102,b837,32.0,21
103,m873,35.0,22
104,fb98,20.0,25
...,...,...,...
1959953,t255,18.0,1
1959957,t139,32.0,1
1959968,w246,20.0,1
1959970,w255,19.0,1


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [12]:
X_train_player_id = X_train['player_id']
X_test_player_id = X_test['player_id']


X_train = X_train.drop('player_id',axis=1)
X_test = X_test.drop('player_id',axis=1)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [14]:
from sklearn.svm import LinearSVR
linear_svr = LinearSVR()
linear_svr.fit(X_train, y_train)

LinearSVR()

In [15]:
# Predicting a new result
y_preds = linear_svr.predict(X_test)

In [16]:
y_preds

array([ 90.01358169,  31.87979053,  23.62528227, ...,  -4.85755052,
       115.4556967 , 100.71180674])

In [17]:
linear_svr.score(X_test, y_test) # r2 score

0.3435998016885755

In [18]:
result = pd.DataFrame({'player_id':X_test_player_id,'pred_ranking_points':y_preds})

In [19]:
result.sort_values('pred_ranking_points',ascending=False,inplace=True)
result.reset_index(drop=True,inplace=True)

In [20]:
result

,player_id,pred_ranking_points
0,a725,244.620453
1,a725,242.176736
2,o482,240.656377
3,a006,238.049482
4,a725,236.447534
...,...,...
91629,ke64,-30.978256
91630,le01,-30.978256
91631,k371,-30.978256
91632,z168,-30.978256


In [21]:
final = result.groupby('player_id').max()

In [22]:
final.reset_index(level=0, inplace=True)

In [23]:
final

,player_id,pred_ranking_points
0,a005,52.271294
1,a006,238.049482
2,a031,18.737849
3,a034,93.217478
4,a041,-7.301267
...,...,...
7771,z429,-4.857551
7772,z443,153.116396
7773,z450,-7.301267
7774,z452,-8.143036


In [24]:
final.sort_values('pred_ranking_points',ascending=False,inplace=True)

In [25]:
final.reset_index(drop=True,inplace=True)

In [26]:
final['Rank'] =  final.index + 1

In [27]:
final.head(10)

,player_id,pred_ranking_points,Rank
0,a725,244.620453,1
1,o482,240.656377,2
2,a006,238.049482,3
3,tc00,233.243638,4
4,s535,229.116384,5
5,a704,227.432846,6
6,z074,225.830898,7
7,la93,223.468771,8
8,n561,222.545413,9
9,s015,220.101696,10
